In [1]:
from pathlib import Path

BASE_DIR = Path.cwd().parent  # Moves up one level from the current notebook's directory



In [2]:
# Этап 1: Загрузка данных с оптимизацией

# Импорт необходимых библиотек
import pandas as pd
import numpy as np


# Указываем путь к данным
# DATA_DIR = "./train"  # Замените на ваш путь
DATA_DIR = BASE_DIR / "data/train"

# Функция для понижения типов данных
def reduce_memory_usage(df):
    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type).startswith('int'):
                if c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
            else:
                df[col] = df[col].astype(np.float32)
    return df

# Загрузка данных

## 1. agg_trades.parquet
print("Загрузка agg_trades.parquet...")
agg_trades_columns = ['instrument', 'ask_amount', 'bid_amount']
agg_trades = pd.read_parquet(f"{DATA_DIR}/agg_trades.parquet", columns=agg_trades_columns)
agg_trades = reduce_memory_usage(agg_trades)

## 2. orderbook_embedding.parquet
print("Загрузка orderbook_embedding.parquet...")
orderbook_embedding_columns = ['instrument', 'mid_price']
orderbook_embedding = pd.read_parquet(f"{DATA_DIR}/orderbook_embedding.parquet", columns=orderbook_embedding_columns)
orderbook_embedding = reduce_memory_usage(orderbook_embedding)

## 3. orderbook_solusdt.parquet
print("Загрузка orderbook_solusdt.parquet...")
lobs_columns = ['asks[0].price', 'bids[0].price', 'asks[0].amount', 'bids[0].amount']
lobs = pd.read_parquet(f"{DATA_DIR}/orderbook_solusdt.parquet", columns=lobs_columns)
lobs = reduce_memory_usage(lobs)

## 4. target_data_solusdt.parquet
print("Загрузка target_data_solusdt.parquet...")
target_data_columns = ['price', 'side']
target_data = pd.read_parquet(f"{DATA_DIR}/target_data_solusdt.parquet", columns=target_data_columns)
target_data = reduce_memory_usage(target_data)

## 5. target_solusdt.parquet
print("Загрузка target_solusdt.parquet...")
target_columns = ['target']
target = pd.read_parquet(f"{DATA_DIR}/target_solusdt.parquet", columns=target_columns)
target = reduce_memory_usage(target)

# Конвертируем индексы в datetime для согласования
print("Конвертация индексов в datetime...")
for df in [agg_trades, orderbook_embedding, lobs, target_data, target]:
    df.index = pd.to_datetime(df.index)

print("Этап 1 завершён: данные загружены и оптимизированы.")

Загрузка agg_trades.parquet...
Загрузка orderbook_embedding.parquet...
Загрузка orderbook_solusdt.parquet...
Загрузка target_data_solusdt.parquet...
Загрузка target_solusdt.parquet...
Конвертация индексов в datetime...
Этап 1 завершён: данные загружены и оптимизированы.


In [3]:
# Шаг 1: Обработка lobs и вычисление признаков
print("Обработка lobs и вычисление признаков...")

# Вычисляем mid_price
lobs['mid_price'] = (lobs['asks[0].price'] + lobs['bids[0].price']) / 2

# Вычисляем imbalance
lobs['imbalance'] = (
    (lobs['bids[0].amount'] - lobs['asks[0].amount']) /
    (lobs['bids[0].amount'] + lobs['asks[0].amount'] + 1e-8)
)

# Сохраняем вычисленные признаки
lobs_features = lobs[['mid_price', 'imbalance']]

# Освобождаем память, удаляя ненужные столбцы
lobs.drop(columns=['asks[0].price', 'bids[0].price', 'asks[0].amount', 'bids[0].amount'], inplace=True)

# Сохраняем признаки в файл
lobs_features.to_parquet('lobs_features.parquet')

# Освобождаем память
del lobs, lobs_features

print("Признаки из lobs рассчитаны и сохранены.")

Обработка lobs и вычисление признаков...


Признаки из lobs рассчитаны и сохранены.


In [4]:
# Шаг 2: Обработка agg_trades и вычисление признаков
print("Обработка agg_trades и вычисление признаков...")

# Фильтруем данные по инструменту SOLUSDT
solusdt_agg_trades = agg_trades[agg_trades['instrument'] == 'SOLUSDT'].copy()

# Сортируем индекс для корректной работы rolling
solusdt_agg_trades.sort_index(inplace=True)

# Вычисляем sells и buys
sells = solusdt_agg_trades['ask_amount'].rolling(window=10, min_periods=1).sum()
buys = solusdt_agg_trades['bid_amount'].rolling(window=10, min_periods=1).sum()

# Вычисляем trades_ratio
solusdt_agg_trades['trades_ratio'] = (sells - buys) / (sells + buys + 1e-8)

# Сохраняем признак
agg_trades_features = solusdt_agg_trades[['trades_ratio']]

# Сохраняем признаки в файл
agg_trades_features.to_parquet('agg_trades_features.parquet')

# Освобождаем память
del agg_trades, solusdt_agg_trades, sells, buys, agg_trades_features

print("Признаки из agg_trades рассчитаны и сохранены.")

Обработка agg_trades и вычисление признаков...
Признаки из agg_trades рассчитаны и сохранены.


In [5]:
# Шаг 3: Обработка orderbook_embedding и вычисление признаков
print("Обработка orderbook_embedding и вычисление признаков...")

# Извлекаем mid_price для BTCUSDT и ETHUSDT
btcusdt_mid_price = orderbook_embedding[orderbook_embedding['instrument'] == 'BTCUSDT'][['mid_price']]
ethusdt_mid_price = orderbook_embedding[orderbook_embedding['instrument'] == 'ETHUSDT'][['mid_price']]

# Сохраняем их в отдельные файлы
btcusdt_mid_price.to_parquet('btcusdt_mid_price.parquet')
ethusdt_mid_price.to_parquet('ethusdt_mid_price.parquet')

# Освобождаем память
del orderbook_embedding, btcusdt_mid_price, ethusdt_mid_price

print("Признаки из orderbook_embedding рассчитаны и сохранены.")

Обработка orderbook_embedding и вычисление признаков...
Признаки из orderbook_embedding рассчитаны и сохранены.


In [7]:
# Шаг 4: Слияние признаков и подготовка данных для обучения
print("Слияние признаков и подготовка данных для обучения...")

# Загрузка ранее сохранённых признаков
lobs_features = pd.read_parquet('lobs_features.parquet')
agg_trades_features = pd.read_parquet('agg_trades_features.parquet')
btcusdt_mid_price = pd.read_parquet('btcusdt_mid_price.parquet')
ethusdt_mid_price = pd.read_parquet('ethusdt_mid_price.parquet')

# Убедимся, что индексы имеют тип datetime
for df in [lobs_features, agg_trades_features, btcusdt_mid_price, ethusdt_mid_price, target_data, target]:
    df.index = pd.to_datetime(df.index)

# Вычисляем main_btcusdt_dev и main_ethusdt_dev
main_btcusdt_dev = (
    lobs_features['mid_price'] / (btcusdt_mid_price['mid_price'].asof(lobs_features.index) + 1e-8)
).asof(target_data.index) * target_data['side']
main_btcusdt_dev.name = 'main_btcusdt_dev'

main_ethusdt_dev = (
    lobs_features['mid_price'] / (ethusdt_mid_price['mid_price'].asof(lobs_features.index) + 1e-8)
).asof(target_data.index) * target_data['side']
main_ethusdt_dev.name = 'main_ethusdt_dev'

# Вычисляем distance_to_mid_price
distance_to_mid_price = (
    target_data['price'] / (lobs_features['mid_price'].asof(target_data.index) + 1e-8) - 1
) * target_data['side']
distance_to_mid_price.name = 'distance_to_mid_price'

feature_imbalance = lobs_features['imbalance'].asof(target_data.index) * target_data['side']
feature_imbalance.name = "feature_imbalance"

feature_agg_trades_feature = agg_trades_features['trades_ratio'].asof(target_data.index) * target_data['side']
feature_agg_trades_feature.name = "feature_agg_trades_feature"

# Собираем все признаки
features = pd.concat([
    target_data['side'].rename("side"),
    feature_imbalance,
    feature_agg_trades_feature,
    distance_to_mid_price,
    main_ethusdt_dev,
    main_btcusdt_dev
], axis=1)

# Добавляем целевую переменную
features['target'] = target['target'].reindex(features.index)

# Удаляем пропущенные значения
features.dropna(inplace=True)

# Освобождаем память
del lobs_features, agg_trades_features, btcusdt_mid_price, ethusdt_mid_price, target_data, target

print("Признаки объединены и готовы к обучению модели.")

Слияние признаков и подготовка данных для обучения...


Признаки объединены и готовы к обучению модели.


In [8]:
# Шаг 5: Обучение модели
print("Обучение модели...")

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

# Разделение данных на признаки и целевую переменную
X = features.drop(columns=['target'])
y = features['target']

# Освобождаем память, удаляя features, если она больше не нужна
del features

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Освобождаем память, удаляя X и y
del X, y

# Инициализация и обучение модели CatBoostClassifier
model = CatBoostClassifier(**{
    'use_best_model': True,
    'eval_metric': 'Logloss',
    'verbose': 50,
    'iterations': 1200,
    'thread_count': 5,
    'loss_function': 'Logloss',
    'l2_leaf_reg': 50,
    'task_type': 'CPU',
    'depth': 5,
    'learning_rate': 0.01
})

# Обучение модели
model.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    use_best_model=True
)

# Сохранение обученной модели в файл .cbm
model.save_model("model_final.cbm", format="cbm")

# Освобождение памяти
del X_train, X_test, y_train, y_test, model

print("Модель обучена и сохранена в файл 'model_final.cbm'.")

Обучение модели...


0:	learn: 0.6908851	test: 0.6929076	best: 0.6929076 (0)	total: 342ms	remaining: 6m 50s
50:	learn: 0.6848546	test: 0.6849733	best: 0.6849733 (50)	total: 13.5s	remaining: 5m 3s
100:	learn: 0.6815194	test: 0.6816847	best: 0.6816847 (100)	total: 26.7s	remaining: 4m 50s
150:	learn: 0.6801296	test: 0.6802114	best: 0.6802114 (150)	total: 39.4s	remaining: 4m 34s
200:	learn: 0.6793861	test: 0.6794485	best: 0.6794485 (200)	total: 52.2s	remaining: 4m 19s
250:	learn: 0.6788698	test: 0.6789939	best: 0.6789939 (250)	total: 1m 5s	remaining: 4m 5s
300:	learn: 0.6786353	test: 0.6786726	best: 0.6786726 (300)	total: 1m 18s	remaining: 3m 53s
350:	learn: 0.6782712	test: 0.6784140	best: 0.6784140 (350)	total: 1m 31s	remaining: 3m 40s
400:	learn: 0.6780977	test: 0.6782081	best: 0.6782081 (400)	total: 1m 44s	remaining: 3m 28s
450:	learn: 0.6779358	test: 0.6780413	best: 0.6780413 (450)	total: 1m 57s	remaining: 3m 15s
500:	learn: 0.6777915	test: 0.6778949	best: 0.6778949 (500)	total: 2m 10s	remaining: 3m 1s
550